# 02 — Memory Module: MLP Là Bộ Nhớ Dài Hạn

Trong Titans, bộ nhớ dài hạn **không phải vector cố định** mà là **trọng số của một mạng MLP**.

**Ý tưởng:**
- **Trọng số MLP = bộ nhớ** — thông tin được lưu trong cách MLP biến đổi input
- **Đọc bộ nhớ** = forward pass qua MLP
- **Ghi bộ nhớ** = cập nhật trọng số MLP
- **Surprise gating**: chỉ ghi khi surprise cao (effective_lr = write_lr × surprise_score)
- **Forgetting**: weight decay để quên thông tin cũ

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from titans_memory import MemoryModule

torch.manual_seed(42)
mem = MemoryModule(input_dim=4, memory_dim=16, forget_rate=0.0, write_lr=0.01)

x = torch.randn(4)
print("=== Đọc bộ nhớ (trước khi ghi) ===")
output_before = mem.read(x)
print(f"Memory output: {output_before.tolist()}")

print("\n=== Ghi với surprise THẤP (score=0.0) ===")
mem.write(x, surprise_score=torch.tensor(0.0))
output_after_low = mem.read(x)
print(f"Memory output: {output_after_low.tolist()}")
print(f"Thay đổi: {torch.allclose(output_before, output_after_low)} (không đổi)")

print("\n=== Ghi với surprise CAO (score=1.0) ===")
mem.write(x, surprise_score=torch.tensor(1.0))
output_after_high = mem.read(x)
print(f"Memory output: {output_after_high.tolist()}")
print(f"Thay đổi: {not torch.allclose(output_before, output_after_high)} (đã thay đổi!)")

## Surprise-Gated Writes

Công thức cập nhật: `effective_lr = write_lr × surprise_score`

- `surprise_score ≈ 0` → `effective_lr ≈ 0` → **không cập nhật** (đã biết rồi)
- `surprise_score = 1.0` → `effective_lr = write_lr` → **cập nhật mạnh** (thông tin mới!)

Điều này giống cách bạn học: đọc lại điều đã biết thì không nhớ thêm gì, nhưng gặp thông tin hoàn toàn mới thì ghi nhớ ngay.

In [ ]:
torch.manual_seed(42)
mem = MemoryModule(input_dim=1, memory_dim=8, forget_rate=0.0, write_lr=0.05)

# Ghi nhiều lần với các surprise score khác nhau, theo dõi weight thay đổi
surprise_levels = [0.0, 0.1, 0.5, 1.0, 0.0, 0.0, 2.0, 0.0]
weight_changes = []
prev_weights = [p.data.clone() for p in mem.memory_net.parameters()]

for score in surprise_levels:
    x = torch.tensor([3.14])
    mem.write(x, surprise_score=torch.tensor(score))
    curr_weights = [p.data.clone() for p in mem.memory_net.parameters()]
    change = sum((c - p).abs().sum().item() for c, p in zip(curr_weights, prev_weights))
    weight_changes.append(change)
    prev_weights = curr_weights

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 5), sharex=True)
ax1.bar(range(len(surprise_levels)), surprise_levels, color="#FF9800", alpha=0.8)
ax1.set_ylabel("Surprise Score")
ax1.set_title("Surprise Score vs Memory Weight Changes")

ax2.bar(range(len(weight_changes)), weight_changes, color="#4CAF50", alpha=0.8)
ax2.set_ylabel("Tổng |Δweight|")
ax2.set_xlabel("Write step")

plt.tight_layout()
plt.show()

## Cơ Chế Quên (Forgetting)

Memory cũng cần **quên** — nếu không, thông tin cũ sẽ tích tụ và "lấn át" thông tin mới.

Titans dùng **weight decay thích ứng**: `weights *= (1 - forget_rate)` mỗi timestep.

- `forget_rate = 0.0` → không quên (nhớ mãi)
- `forget_rate = 0.05` → quên nhanh (chỉ nhớ gần đây)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

for rate in [0.0, 0.005, 0.02, 0.05]:
    torch.manual_seed(42)
    mem = MemoryModule(input_dim=1, memory_dim=16, forget_rate=rate, write_lr=0.01)

    # Ghi 1 lần rồi theo dõi memory norm giảm dần
    mem.write(torch.tensor([5.0]), surprise_score=torch.tensor(1.0))
    norms = []
    for _ in range(100):
        norm = sum(p.data.norm().item() for p in mem.memory_net.parameters())
        norms.append(norm)
        mem.apply_forgetting()

    ax.plot(norms, linewidth=2, label=f"forget_rate={rate}")

ax.set_xlabel("Timestep sau khi ghi")
ax.set_ylabel("Tổng Memory Weight Norm")
ax.set_title("Hiệu ứng Forgetting: Weight decay theo thời gian")
ax.legend()
plt.tight_layout()
plt.show()

## Kết luận

- **MLP weights = bộ nhớ**: cách tiếp cận sáng tạo, khác hoàn toàn RNN truyền thống
- **Surprise gating**: chỉ ghi nhớ khi thông tin thực sự mới lạ
- **Forgetting**: tự động loại bỏ thông tin cũ, giữ bộ nhớ "tươi mới"

→ Ba cơ chế này kết hợp tạo nên hệ thống bộ nhớ dài hạn thông minh.